# Densest Point Neighborhood Analysis

**Goal:** Explore the token at the highest density point and its neighborhood.

**Method:**
1. Find token with maximum k-NN density
2. Get k nearest neighbors in **causal metric** (using precomputed distances)
3. Get k nearest neighbors in **cosine similarity** (raw embeddings)
4. Compare: Do causal metric neighborhoods = semantic neighborhoods?
5. Decode tokens to see what they actually are

**Key question:** Does the causal metric capture semantic structure, or something else?

**Expected runtime:** ~2 minutes

## Configuration

In [1]:
# Input files
DISTANCES_PATH = '../data/vectors/distances_causal_32000_full.npy'
METADATA_PATH = '../data/vectors/distances_causal_32000.pt'
METRIC_TENSOR_PATH = '../data/vectors/causal_metric_tensor_qwen3_4b.pt'

# Model
MODEL_NAME = 'Qwen/Qwen3-4B-Instruct-2507'

# Analysis parameters
K_NEIGHBORS = 20  # Number of neighbors to examine
TOP_N = 30        # Show top N neighbors

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  k-NN: {K_NEIGHBORS} neighbors")
print(f"  Show top: {TOP_N} neighbors")

Configuration:
  Model: Qwen/Qwen3-4B-Instruct-2507
  k-NN: 20 neighbors
  Show top: 30 neighbors


## Setup

In [2]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

print("✓ Imports complete")

✓ Imports complete


## Load Data

In [3]:
print("Loading distance matrix...")
distances = np.load(DISTANCES_PATH)
N = distances.shape[0]

print("Loading metadata...")
metadata = torch.load(METADATA_PATH, weights_only=False)
token_indices = metadata['token_indices'].numpy()

print(f"\n✓ Loaded data")
print(f"  {N:,} tokens in sample")
print(f"  Token indices: [{token_indices.min()}, {token_indices.max()}]")

Loading distance matrix...
Loading metadata...

✓ Loaded data
  32,000 tokens in sample
  Token indices: [5, 151930]


## Compute k-NN Densities

In [4]:
print("\nComputing k-NN densities...")

densities = np.zeros(N)
mean_distances = np.zeros(N)

for i in range(N):
    dists_i = distances[i]
    k_nearest = np.partition(dists_i, K_NEIGHBORS)[:K_NEIGHBORS+1]
    k_nearest = k_nearest[k_nearest > 0][:K_NEIGHBORS]
    
    mean_dist = k_nearest.mean()
    mean_distances[i] = mean_dist
    densities[i] = 1.0 / mean_dist if mean_dist > 0 else 0

print(f"✓ Computed densities")
print(f"\nDensity statistics:")
print(f"  Min: {densities.min():.6f}")
print(f"  Max: {densities.max():.6f}")
print(f"  Mean: {densities.mean():.6f}")


Computing k-NN densities...


/var/folders/k5/44vd1ct56xj4y9h7x213kvjr0000gn/T/ipykernel_27030/171613569.py:11: RuntimeWarning: Mean of empty slice.
  mean_dist = k_nearest.mean()
/Users/jefferyharrell/Projects/Azimuth/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


✓ Computed densities

Density statistics:
  Min: 0.000000
  Max: 2413.602051
  Mean: 0.595077


## Find Densest Point

In [5]:
print("\n" + "=" * 70)
print("DENSEST POINT ANALYSIS")
print("=" * 70)

# Find densest token in our sample
densest_idx = np.argmax(densities)
densest_token_id = token_indices[densest_idx]
densest_density = densities[densest_idx]
densest_mean_dist = mean_distances[densest_idx]

print(f"\nDensest point:")
print(f"  Sample index: {densest_idx}")
print(f"  Vocabulary token ID: {densest_token_id}")
print(f"  Density: {densest_density:.6f} (1/logometers)")
print(f"  Mean distance to {K_NEIGHBORS} neighbors: {densest_mean_dist:.4f} logometers")


DENSEST POINT ANALYSIS

Densest point:
  Sample index: 734
  Vocabulary token ID: 141503
  Density: 2413.602051 (1/logometers)
  Mean distance to 20 neighbors: 0.0004 logometers


## Load Model and Tokenizer

In [6]:
print("\nLoading model and tokenizer...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map='cpu',
)

gamma = model.lm_head.weight.data
del model

print(f"✓ Loaded model")
print(f"  Vocabulary size: {gamma.shape[0]:,}")
print(f"  Embedding dimension: {gamma.shape[1]:,}")


Loading model and tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✓ Loaded model
  Vocabulary size: 151,936
  Embedding dimension: 2,560


## Extract Sampled Embeddings

In [7]:
print("\nExtracting embeddings for sampled tokens...")

sampled_embeddings = gamma[token_indices]  # [N, hidden_dim]

print(f"✓ Extracted embeddings")
print(f"  Shape: {sampled_embeddings.shape}")


Extracting embeddings for sampled tokens...
✓ Extracted embeddings
  Shape: torch.Size([32000, 2560])


## Find k-NN in Causal Metric

These are already computed! Just sort the distance matrix row.

In [8]:
print("\nFinding k nearest neighbors in CAUSAL METRIC...")

# Get distances from densest point to all others
causal_dists = distances[densest_idx]

# Sort to get nearest neighbors (excluding self at distance 0)
causal_sorted = np.argsort(causal_dists)
causal_knn_indices = causal_sorted[1:TOP_N+1]  # Skip self (index 0)
causal_knn_dists = causal_dists[causal_knn_indices]
causal_knn_token_ids = token_indices[causal_knn_indices]

print(f"✓ Found {TOP_N} nearest neighbors in causal metric")
print(f"  Distance range: [{causal_knn_dists.min():.4f}, {causal_knn_dists.max():.4f}] logometers")


Finding k nearest neighbors in CAUSAL METRIC...
✓ Found 30 nearest neighbors in causal metric
  Distance range: [0.0000, 0.0004] logometers


## Find k-NN in Cosine Similarity

Compute cosine similarity to all tokens in sample.

In [9]:
print("\nFinding k nearest neighbors in COSINE SIMILARITY...")

# Get embedding of densest point
densest_embedding = sampled_embeddings[densest_idx]  # [hidden_dim]

# Compute cosine similarities to all sampled tokens
# cosine_sim = (u · v) / (||u|| ||v||)
norms = torch.norm(sampled_embeddings, dim=1)  # [N]
densest_norm = torch.norm(densest_embedding)

dot_products = sampled_embeddings @ densest_embedding  # [N]
cosine_sims = dot_products / (norms * densest_norm)
cosine_sims = cosine_sims.numpy()

# Sort by similarity (highest first)
cosine_sorted = np.argsort(cosine_sims)[::-1]
cosine_knn_indices = cosine_sorted[1:TOP_N+1]  # Skip self (similarity = 1.0)
cosine_knn_sims = cosine_sims[cosine_knn_indices]
cosine_knn_token_ids = token_indices[cosine_knn_indices]

print(f"✓ Found {TOP_N} nearest neighbors in cosine similarity")
print(f"  Similarity range: [{cosine_knn_sims.min():.4f}, {cosine_knn_sims.max():.4f}]")


Finding k nearest neighbors in COSINE SIMILARITY...
✓ Found 30 nearest neighbors in cosine similarity
  Similarity range: [1.0000, 1.0000]


## Decode Tokens

Convert token IDs to actual text.

In [10]:
def decode_token(token_id):
    """Decode a single token ID to text."""
    try:
        text = tokenizer.decode([token_id])
        # Clean up: show repr for special chars
        if text.strip() == '':
            return repr(text)
        return text
    except:
        return "<ERROR>"

# Decode densest token
densest_text = decode_token(densest_token_id)

# Decode causal neighbors
causal_texts = [decode_token(tid) for tid in causal_knn_token_ids]

# Decode cosine neighbors
cosine_texts = [decode_token(tid) for tid in cosine_knn_token_ids]

print(f"\n✓ Decoded tokens")


✓ Decoded tokens


## Display Results: Densest Token

In [11]:
print("\n" + "=" * 70)
print("DENSEST TOKEN")
print("=" * 70)

print(f"\nToken ID: {densest_token_id}")
print(f"Text: {densest_text}")
print(f"Density: {densest_density:.6f} (1/logometers)")
print(f"Mean neighbor distance: {densest_mean_dist:.4f} logometers")


DENSEST TOKEN

Token ID: 141503
Text: การบริหาร
Density: 2413.602051 (1/logometers)
Mean neighbor distance: 0.0004 logometers


## Display Results: Causal Metric Neighbors

In [12]:
print("\n" + "=" * 70)
print(f"TOP {TOP_N} NEIGHBORS: CAUSAL METRIC (Distance-based)")
print("=" * 70)

causal_df = pd.DataFrame({
    'Rank': range(1, TOP_N + 1),
    'Token ID': causal_knn_token_ids,
    'Distance': causal_knn_dists,
    'Text': causal_texts
})

print("\n" + causal_df.to_string(index=False))


TOP 30 NEIGHBORS: CAUSAL METRIC (Distance-based)

 Rank  Token ID  Distance          Text
    1       180  0.000000             �
    2    138979  0.000000        รับรอง
    3    124707  0.000345            ต์
    4    134995  0.000345        เสี่ยง
    5    123848  0.000423             �
    6    127321  0.000423          นั่น
    7    131745  0.000423          ล้าน
    8    129342  0.000423           ใส่
    9    126637  0.000423        เกี่ยว
   10    126668  0.000423          ยิ่ง
   11    132276  0.000423         พิมพ์
   12    147628  0.000423             類
   13    142915  0.000423          ตุลา
   14    132108  0.000423       ป้องกัน
   15    132098  0.000423     เทคโนโลยี
   16    149987  0.000423             梁
   17    132715  0.000423          พนัน
   18    136380  0.000423        ทุกวัน
   19    130159  0.000423            فَ
   20    149257  0.000423             輪
   21    126324  0.000423          เป็น
   22    130711  0.000423         ศูนย์
   23    143257  0.000423 คาส

## Display Results: Cosine Similarity Neighbors

In [13]:
print("\n" + "=" * 70)
print(f"TOP {TOP_N} NEIGHBORS: COSINE SIMILARITY (Embedding-based)")
print("=" * 70)

cosine_df = pd.DataFrame({
    'Rank': range(1, TOP_N + 1),
    'Token ID': cosine_knn_token_ids,
    'Similarity': cosine_knn_sims,
    'Text': cosine_texts
})

print("\n" + cosine_df.to_string(index=False))


TOP 30 NEIGHBORS: COSINE SIMILARITY (Embedding-based)

 Rank  Token ID  Similarity           Text
    1       180    1.000002              �
    2    138979    1.000002         รับรอง
    3    124707    1.000002             ต์
    4    141503    1.000002      การบริหาร
    5    132709    1.000002          ว่าจะ
    6    138035    1.000002            รีบ
    7    151869    1.000002             ''
    8    131078    1.000002         เข้าไป
    9    151814    1.000002             ''
   10    135773    1.000002       การพัฒนา
   11    124212    1.000002           ื่อง
   12    124484    1.000002             นั
   13    151654    1.000002 <|vision_pad|>
   14    137681    1.000002          ชัดเจ
   15    141155    1.000002             ผู
   16    124383    1.000002           ต้อง
   17    140928    1.000002            บัง
   18    132742    1.000002    เจ้าหน้าที่
   19    128190    1.000002             ศิ
   20    136561    1.000002          อาศัย
   21    126770    1.000002            พั

## Comparison: Overlap Between Neighborhoods

In [14]:
print("\n" + "=" * 70)
print("NEIGHBORHOOD OVERLAP ANALYSIS")
print("=" * 70)

# Find tokens that appear in both neighborhoods
causal_set = set(causal_knn_token_ids)
cosine_set = set(cosine_knn_token_ids)
overlap = causal_set & cosine_set

overlap_pct = 100 * len(overlap) / TOP_N

print(f"\nTokens in both neighborhoods: {len(overlap)} / {TOP_N} ({overlap_pct:.1f}%)")

if len(overlap) > 0:
    print(f"\nOverlapping tokens:")
    for token_id in overlap:
        text = decode_token(token_id)
        causal_rank = np.where(causal_knn_token_ids == token_id)[0][0] + 1
        cosine_rank = np.where(cosine_knn_token_ids == token_id)[0][0] + 1
        print(f"  {token_id:6d}: {text:20s} (causal rank: {causal_rank:2d}, cosine rank: {cosine_rank:2d})")

print(f"\n" + "=" * 70)
print("INTERPRETATION:")
print("=" * 70)

if overlap_pct > 70:
    print(f"\n✅ HIGH OVERLAP ({overlap_pct:.1f}%)")
    print("   Causal metric neighborhoods ≈ semantic neighborhoods")
    print("   The causal metric captures semantic structure!")
elif overlap_pct > 30:
    print(f"\n⚠️  MODERATE OVERLAP ({overlap_pct:.1f}%)")
    print("   Some agreement between causal and cosine metrics")
    print("   Causal metric partially captures semantic structure")
else:
    print(f"\n❌ LOW OVERLAP ({overlap_pct:.1f}%)")
    print("   Causal metric neighborhoods ≠ semantic neighborhoods")
    print("   The causal metric captures different structure than raw embeddings!")


NEIGHBORHOOD OVERLAP ANALYSIS

Tokens in both neighborhoods: 4 / 30 (13.3%)

Overlapping tokens:
  124707: ต์                   (causal rank:  3, cosine rank:  3)
     180: �                    (causal rank:  1, cosine rank:  1)
  132581: เดียวกัน             (causal rank: 28, cosine rank: 24)
  138979: รับรอง               (causal rank:  2, cosine rank:  2)

INTERPRETATION:

❌ LOW OVERLAP (13.3%)
   Causal metric neighborhoods ≠ semantic neighborhoods
   The causal metric captures different structure than raw embeddings!


## Summary

**Question:** Does the causal metric capture semantic neighborhoods?

**Method:** Compare k-NN in causal metric vs cosine similarity for densest point

**Answer:** Check the overlap percentage and decoded tokens above!

---

**Key insights:**
- Densest point likely in r~22 cluster (from 07.54)
- Causal neighbors: found via distance in M-metric
- Cosine neighbors: found via raw embedding similarity
- Overlap reveals if causal metric = semantic structure